In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Audiobooks_data.csv to Audiobooks_data.csv


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn import preprocessing

In [ ]:
raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter=',')

#The inputs are all columns in the csv, except for the first one [:,0]
# and the last one [:,-1] (which is our targets)
unscaled_inputs_all=raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all=raw_csv_data[:,-1]

In [ ]:
##Balance the Dataset

# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets=int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter=0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
indices_to_remove=[]

#Count the no. of targets that're 0
#Once there're as many 0s as 1s, mark entries where the target is 0
for i in range(targets_all.shape[0]):
  if targets_all[i]==0:
    zero_targets_counter+=1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

#Create 2 new variables, one that will contain the inputs,& one that will contain the targets
# We delete all indices that we amrkes "to remove" in the loop above

unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [ ]:
##Standardize the inputs

# That's the only place we use sklearn functionality. We will take advantage of its preprocessing capabilities
# It's a simple line of code, which standardizes the inputs, as we explained in one of the lectures.
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

In [ ]:
#Shuffle the data

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]

In [ ]:
##Split the dataset into train,validation & test

samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1810.0 3579 0.5057278569432803
211.0 447 0.4720357941834452
216.0 448 0.48214285714285715


In [ ]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)